## **ABOUT**
This example illustrates the 2D multi-slice, Spin Echo (SE) acquisition using the `pypulseq` library. This sequence is typically used for T<sub>2</sub> weighted imaging. A 2D Fourier transform can be used to reconstruct images from this acquisition. Read more about SE [here](http://mriquestions.com/se-vs-multi-se-vs-fse.html).

**Contact**: For issues, write to ks3621@columbia.edu

---

## **INSTALL** `pypulseq`

In [0]:
!pip install pypulseq

## **IMPORT PACKAGES**

In [0]:
import matplotlib as mpl
mpl.use('TkAgg')

In [0]:
from math import pi

import numpy as np

from pypulseq.Sequence.sequence import Sequence
from pypulseq.calc_duration import calc_duration
from pypulseq.make_adc import make_adc
from pypulseq.make_delay import make_delay
from pypulseq.make_sinc_pulse import make_sinc_pulse
from pypulseq.make_trap_pulse import make_trapezoid
from pypulseq.opts import Opts

## **USER INPUTS**

These parameters are typically on the user interface of the scanner computer console 

In [0]:
nsa = 1  # Number of averages
n_slices = 3  # Number of slices
Nx = 128
Ny = 128
fov = 220e-3  # mm
slice_thickness = 5e-3  # s
slice_gap = 15e-3  # s
rf_flip = 90  # degrees
rf_offset = 0
print('User inputs setup')

## **SYSTEM LIMITS**
Set the hardware limits and initialize sequence object

In [0]:
system = Opts(max_grad=32, grad_unit='mT/m', max_slew=130, slew_unit='T/m/s', 
              grad_raster_time=10e-6, rf_ringdown_time=10e-6, 
              rf_dead_time=100e-6)
seq = Sequence(system)

## **TIME CONSTANTS**

In [0]:
TE = 100e-3  # s
TR = 3  # s
tau = TE / 2  # s
readout_time = 6.4e-3
pre_time = 8e-4  # s

## **RF**

In [0]:
flip90 = round(rf_flip * pi / 180, 3)
flip180 = 180 * pi / 180
rf90, gz90, _ = make_sinc_pulse(flip_angle=flip90, system=system, duration=4e-3, 
                                slice_thickness=slice_thickness, apodization=0.5, 
                                time_bw_product=4)
rf180, gz180, _ = make_sinc_pulse(flip_angle=flip180, system=system, 
                                  duration=2.5e-3, 
                                  slice_thickness=slice_thickness, 
                                  apodization=0.5, 
                                time_bw_product=4, phase_offset=90 * pi/180)

## **READOUT**
Readout gradients and related events

In [0]:
delta_k = 1 / fov
k_width = Nx * delta_k
gx = make_trapezoid(channel='x', system=system, flat_area=k_width, 
                    flat_time=readout_time)
adc = make_adc(num_samples=Nx, duration=gx.flat_time, delay=gx.rise_time)

## **PREPHASE AND REPHASE**

In [0]:
phase_areas = (np.arange(Ny) - (Ny / 2)) * delta_k
gz_reph = make_trapezoid(channel='z', system=system, area=-gz90.area / 2,
                         duration=2.5e-3)
gx_pre = make_trapezoid(channel='x', system=system, flat_area=k_width / 2, 
                        flat_time=readout_time / 2)
gy_pre = make_trapezoid(channel='y', system=system, area=phase_areas[-1], 
                        duration=2e-3)

## **SPOILER**

In [0]:
gz_spoil = make_trapezoid(channel='z', system=system, area=gz90.area * 4,
                          duration=pre_time * 4)

## **DELAYS**
Echo time (TE) and repetition time (TR). Here, TE is broken down into `delay1` and `delay2`.

In [0]:
delay1 = tau - calc_duration(rf90) / 2 - calc_duration(gx_pre)
delay1 -= calc_duration(gz_spoil) - calc_duration(rf180) / 2
delay1 = make_delay(delay1)
delay2 = tau - calc_duration(rf180) / 2 - calc_duration(gz_spoil)
delay2 -= calc_duration(gx) / 2
delay2 = make_delay(delay2)
delay_TR = TR - calc_duration(rf90) / 2 - calc_duration(gx) / 2 - TE
delay_TR -= calc_duration(gy_pre)
delay_TR = make_delay(delay_TR)
print(f'delay_1: {delay1}')
print(f'delay_2: {delay1}')
print(f'delay_TR: {delay_TR}')

## **CONSTRUCT SEQUENCE**
Construct sequence for one phase encode and multiple slices

In [0]:
# Prepare RF offsets. This is required for multi-slice acquisition
delta_z = n_slices * slice_gap
z = np.linspace((-delta_z / 2), (delta_z / 2), n_slices) + rf_offset

for k in range(nsa):  # Averages
  for j in range(n_slices):  # Slices
    # Apply RF offsets
    freq_offset = gz90.amplitude * z[j]
    rf90.freq_offset = freq_offset

    freq_offset = gz180.amplitude * z[j]
    rf180.freq_offset = freq_offset

    for i in range(Ny):  # Phase encodes
      seq.add_block(rf90, gz90)
      gy_pre = make_trapezoid(channel='y', system=system, 
                              area=phase_areas[-i -1], duration=2e-3)
      seq.add_block(gx_pre, gy_pre, gz_reph)
      seq.add_block(delay1)
      seq.add_block(gz_spoil)
      seq.add_block(rf180, gz180)
      seq.add_block(gz_spoil)
      seq.add_block(delay2)
      seq.add_block(gx, adc)
      gy_pre = make_trapezoid(channel='y', system=system, 
                              area=-phase_areas[-j -1], duration=2e-3)
      seq.add_block(gy_pre, gz_spoil)
      seq.add_block(delay_TR)

## **PLOTTING TIMNG DIAGRAM**

In [0]:
seq.plot(time_range=(0, 0.1))

## **GENERATING `.SEQ` FILE**
Uncomment the code in the cell below to generate a `.seq` file and download locally.

In [0]:
# seq.write('t2_se_pypulseq_colab.seq')  # Save to disk
# from google.colab import files
# files.download('t2_se_pypulseq_colab.seq')  # Download locally